

---


# **Sistema de Detección de Quejas y Soporte Automático para E-commerce 📩**


---



---



-------------------------------------
# 1. **Carga de librerías y dataset**
-------------------------------------

In [1]:
!pip install gradio -q
import gradio as gr

In [2]:
%pip install -q transformers
from transformers import pipeline
import pandas as pd

In [4]:
# Instalar Google Gemini API
%pip install -U -q google-genai
from google.colab import userdata
import os
import re
# Definir la API key de Google Gemini
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

from google import genai
from google.genai import types

# Cliente de la API
cliente = genai.Client(api_key=GOOGLE_API_KEY)

In [5]:
MODEL_ID = "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input":true, isTemplate: true}

In [6]:
# -----------+ Dataset con: productos, lugares, clientes, pedidos +-----------

data = {
    'ticket_text': [
        "Hola, soy Martín Rodríguez de Córdoba. Mi router TP-Link no conecta desde ayer.",
        "¡Excelente atención! La asesora Paula me ayudó a configurar mi televisor LG en minutos.",
        "Compré una notebook Lenovo en la sucursal Palermo y vino con la pantalla rota.",
        "¿Pueden verificar el pedido #89342? Me llegó una cafetera en vez del microondas que pedí.",
        "Solicité asistencia técnica para el aire acondicionado Samsung y no vino nadie.",
        "Tuve un problema con la entrega en Rosario, el paquete llegó una semana tarde.",
        "Me cobraron dos veces la factura del 05/06. Cliente: Federico Gómez.",
        "La app de MercadoPago anda lenta en mi celular Xiaomi. ¿Qué pasa?",
        "Agradezco a Julián del soporte, resolvió mi problema con el horno eléctrico enseguida.",
        "El sitio web está caído desde anoche. No puedo avanzar con la compra.",
        "Compré el combo gamer (teclado + mouse + auriculares) y faltan los auriculares.",
        "El número de orden 101209 aparece como entregado, pero no recibí nada.",
        "Muy conforme con la atención en la tienda de Mendoza. Todo perfecto.",
        "¿Por qué no funciona el seguimiento del pedido #77234 en la app?",
        "Recibí el TV Samsung QLED 55'' en tiempo y forma. ¡Gracias equipo!",
    ],
    'category': [
        "Problema Técnico", "Elogio", "Problema de Producto", "Error de Envío", "Consulta Técnica",
        "Problema de Envío", "Consulta de Facturación", "Problema Técnico", "Elogio",
        "Problema Técnico", "Falta de Producto", "Error de Envío", "Elogio",
        "Consulta de Envío", "Elogio"
    ],
    'sentiment': [
        "Negativo", "Positivo", "Negativo", "Negativo", "Negativo",
        "Negativo", "Negativo", "Negativo", "Positivo",
        "Negativo", "Negativo", "Negativo", "Positivo",
        "Negativo", "Positivo"
    ]
}

import pandas as pd

df_tickets = pd.DataFrame(data)
display(df_tickets)


,ticket_text,category,sentiment
0,"Hola, soy Martín Rodríguez de Córdoba. Mi rout...",Problema Técnico,Negativo
1,¡Excelente atención! La asesora Paula me ayudó...,Elogio,Positivo
2,Compré una notebook Lenovo en la sucursal Pale...,Problema de Producto,Negativo
3,¿Pueden verificar el pedido #89342? Me llegó u...,Error de Envío,Negativo
4,Solicité asistencia técnica para el aire acond...,Consulta Técnica,Negativo
5,"Tuve un problema con la entrega en Rosario, el...",Problema de Envío,Negativo
6,Me cobraron dos veces la factura del 05/06. Cl...,Consulta de Facturación,Negativo
7,La app de MercadoPago anda lenta en mi celular...,Problema Técnico,Negativo
8,"Agradezco a Julián del soporte, resolvió mi pr...",Elogio,Positivo
9,El sitio web está caído desde anoche. No puedo...,Problema Técnico,Negativo




---


# 2. **Detección de Sentimiento y Clasificación Zero-Shot:**


---





---


 **--------------------------------Análisis de Sentimiento con BETO--------------------------------**


---



In [ ]:
# Cargamos el modelo de análisis de sentimiento en español
beto_sentiment = pipeline("sentiment-analysis", model="finiteautomata/beto-sentiment-analysis")

# Aplicamos el modelo a cada ticket para obtener la polaridad
df_tickets['ticket_beto'] = df_tickets['ticket_text'].apply(lambda x: beto_sentiment(x)[0]['label'])
df_tickets['score_beto'] = df_tickets['ticket_text'].apply(lambda x: beto_sentiment(x)[0]['score'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/481k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


+ **ticket_beto -->** *Etiqueta de sentimiento (POS, NEG, NEU)*
+  **score_beto -->** *Confianza del modelo para esa etiqueta*



---


 **--------------------------------Clasificación Zero-Shot (Categoría del ticket)--------------------------------**


---



In [ ]:
# Cargamos el modelo zero-shot para español
zero_shot = pipeline("zero-shot-classification", model="Recognai/zeroshot_selectra_medium")

# Etiquetas de categorías posibles (usamos las manuales del dataset)
candidate_labels = df_tickets['category'].unique().tolist()

# Aplicamos clasificación zero-shot para cada ticket
df_tickets['ticket_zero'] = df_tickets['ticket_text'].apply(lambda x: zero_shot(x, candidate_labels=candidate_labels)['labels'][0])
df_tickets['score_zero'] = df_tickets['ticket_text'].apply(lambda x: zero_shot(x, candidate_labels=candidate_labels)['scores'][0])

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/163M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/387k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


+ **ticket_zero -->** *categoría predicha*
+  **score_zero -->** *confianza del modelo para esa categoría*



---



 **--------------------------------Análisis de Resultados --------------------------------**


---



In [ ]:
# -----------+ comparación para revisar discrepancias +-----------

# Diccionario para traducir etiquetas del modelo BETO a etiquetas humanas
beto_map = {"POS": "Positivo", "NEG": "Negativo", "NEU": "Neutral"}

# Creamos nueva columna con las etiquetas traducidas de BETO
df_tickets['ticket_beto_traducido'] = df_tickets['ticket_beto'].map(beto_map)

# Comparamos el sentimiento manual con el predicho (traducido) por BETO
df_tickets['coincide_sentimiento'] = df_tickets['sentiment'] == df_tickets['ticket_beto_traducido']

# Comparamos la categoría manual con la predicha por Zero-Shot
df_tickets['coincide_categoria'] = df_tickets['category'] == df_tickets['ticket_zero']

# Mostramos las columnas clave para análisis
display(df_tickets[[
    'ticket_text',                # Texto original del ticket
    'sentiment',                  # Sentimiento manual
    'ticket_beto_traducido',      # Sentimiento predicho por BETO (traducido)
    'coincide_sentimiento',       # ¿Coinciden los sentimientos?
    'category',                   # Categoría manual
    'ticket_zero',                # Categoría predicha por Zero-Shot
    'coincide_categoria'          # ¿Coinciden las categorías?
]])

,ticket_text,sentiment,ticket_beto_traducido,coincide_sentimiento,category,ticket_zero,coincide_categoria
0,"Hola, soy Martín Rodríguez de Córdoba. Mi rout...",Negativo,Neutral,False,Problema Técnico,Elogio,False
1,¡Excelente atención! La asesora Paula me ayudó...,Positivo,Positivo,True,Elogio,Elogio,True
2,Compré una notebook Lenovo en la sucursal Pale...,Negativo,Neutral,False,Problema de Producto,Falta de Producto,False
3,¿Pueden verificar el pedido #89342? Me llegó u...,Negativo,Neutral,False,Error de Envío,Consulta Técnica,False
4,Solicité asistencia técnica para el aire acond...,Negativo,Negativo,True,Consulta Técnica,Consulta Técnica,True
5,"Tuve un problema con la entrega en Rosario, el...",Negativo,Negativo,True,Problema de Envío,Problema de Envío,True
6,Me cobraron dos veces la factura del 05/06. Cl...,Negativo,Neutral,False,Consulta de Facturación,Consulta de Facturación,True
7,La app de MercadoPago anda lenta en mi celular...,Negativo,Negativo,True,Problema Técnico,Falta de Producto,False
8,"Agradezco a Julián del soporte, resolvió mi pr...",Positivo,Neutral,False,Elogio,Problema Técnico,False
9,El sitio web está caído desde anoche. No puedo...,Negativo,Negativo,True,Problema Técnico,Falta de Producto,False




---


**--------------------------------Rendimiento General--------------------------------**


---

**----------------- BETO (Sentimiento) -----------------**
 +   Métrica Evaluada --> Coincidencia con etiquetas manuales
 + Precisión Obtenida: 53.3% (8/15 aciertos)

*BETO acertó solo la mitad de los casos, especialmente en los sentimientos negativos. Tiene margen de mejora en la detección de sentimientos más neutros o ambiguos.*

**----------------- Zero-Shot (Categoría) -----------------**
 +   Métrica Evaluada --> Coincidencia con etiquetas manuales
 + Precisión Obtenida: 46.7% (7/15 aciertos)

*Zero-Shot mostró un rendimiento más bajo, especialmente al clasificar categorías menos claras. Esto es común en modelos que intentan predecir sin datos contextuales muy específicos*.

# 3. **Extracción de Información Clave (NER y QA)**

In [7]:
# ------------------------------
#  Inicialización de Modelos
# ------------------------------

# Modelo de reconocimiento de entidades (NER), para extraer nombres, ubicaciones, productos, etc.
ner_pipeline = pipeline(
    "ner",  # Tarea: Named Entity Recognition
    model="mrm8488/bert-spanish-cased-finetuned-ner",  # Modelo entrenado en español para detectar entidades
    aggregation_strategy="simple"  # Agrupa tokens contiguos de una misma entidad (ej: "Juan Pérez")
)

# Modelo de pregunta-respuesta (QA), para hacer preguntas a los textos de los tickets
qa_pipeline = pipeline(
    "question-answering",  # Tarea: QA
    model="PlanTL-GOB-ES/roberta-large-bne-sqac"  # Modelo robusto en español, entrenado en preguntas tipo SQuAD
)

# ------------------------------
#  QA Seguro (Validación de Respuestas)
# ------------------------------

def safe_answer(text, question, threshold=10):
    """
    Realiza una pregunta sobre un texto y valida que la respuesta sea útil.
    Evita respuestas vacías, genéricas o irrelevantes.
    """
    try:
        result = qa_pipeline(question=question, context=text)  # Se lanza la pregunta sobre el texto
        answer = result['answer'].strip()  # Se extrae y limpia la respuesta
        score = result.get('score', 1.0)  # Se captura el puntaje de confianza (fallback = 1.0)

        # Si la respuesta es muy corta o es una negación, la descartamos
        if len(answer) < threshold or answer.lower() in ["no", "ninguno", "nada"]:
            return ""
        return answer  # Si pasó todos los filtros, la devolvemos
    except Exception as e:
        print(f"Error en QA: {e}")  # Log de error por consola
        return ""  # Si hubo algún error, devolvemos vacío


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/858k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/516k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
# ------------------------------
#  Extracción de Entidades (NER)
# ------------------------------

# Aplicamos el modelo NER a cada texto de ticket para extraer entidades (nombres, ubicaciones, productos, etc.)
df_tickets['entidades'] = df_tickets['ticket_text'].apply(ner_pipeline)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [9]:
# ------------------------------
#  Filtro de Ruido Post-QA
# ------------------------------

def limpiar_respuesta_qa(respuesta, campo):
    """
    Limpieza avanzada de respuestas QA, con filtros personalizados por campo.
    """
    if not respuesta or not isinstance(respuesta, str):
        return ""

    respuesta = respuesta.strip()

    # 1. Filtrar respuestas sospechosamente cortas o con signos raros
    if len(respuesta.split()) <= 1 or any(char in respuesta for char in ['#', '@', '?']):
        return ""

    # 2. Limpieza por campo específico
    if campo == "ubicacion":
        if df_tickets["entidades"].any()=="LOC":
          return df_tickets["entidades"].values
        # Si contiene verbos o frases genéricas, no es ubicación real
        elif re.search(r"\b(llegó|vino|anda|funciona|entrega|recibí|caído|factura)\b", respuesta.lower()):
            return ""
        if respuesta.lower().startswith("el sitio") or "no puedo" in respuesta.lower():
            return ""
    elif campo == "nombre_cliente":
        if df_tickets["entidades"].any()== "PER":
          return df_tickets["entidades"].values
        # Si el nombre incluye el producto, o es igual al ticket entero, lo descartamos
        elif re.search(r"\b(router|televisor|cafetera|combo|paquete|pedido|orden|producto|teclado)\b", respuesta.lower()):
            return ""
        if len(respuesta) > 50:  # Si se va de mambo
            return ""
    elif campo == "producto_mencionado":
        # Si el producto es una persona o una oración entera
        if re.search(r"\b(federico|gómez|julián|paula)\b", respuesta.lower()):
            return ""
        if len(respuesta.split()) > 6:  # Muy largo = probablemente frase entera
            return ""
    elif campo == "problema_principal":
        # Elimina repeticiones completas del ticket
        if respuesta.lower() in ["no puedo avanzar con la compra", "no recibí nada"]:
            return respuesta  # Excepciones válidas
        if len(respuesta) > 100 or respuesta in respuesta.lower():
            return respuesta[:100].rsplit('.', 1)[0]  # Corta si es muy largo

    return respuesta


In [10]:
# ------------------------------
#  Definición de Preguntas para QA
# ------------------------------

qa_questions = {
    'producto_mencionado': "¿Qué producto se menciona?",  # Queremos saber qué producto está involucrado
    'ubicacion': "¿Dónde ocurrió el problema?",  # Preguntamos por el lugar del incidente
    'nombre_cliente': "¿Cuál es el nombre de la persona que escribió?",  # Queremos capturar el nombre del cliente
    'problema_principal': "¿Cuál es el problema?"  # Identificamos la queja principal
}


In [11]:
# ------------------------------
#  Clasificación de Tickets
# ------------------------------

# Creamos una columna paralela a "category" para trabajar sin tocar el original
df_tickets['tipo_ticket'] = df_tickets['category']


In [12]:
# ------------------------------
#  Aplicar QA y Limpiar Respuestas
# ------------------------------

# Recorremos cada campo objetivo (col) con su pregunta correspondiente
for col, pregunta in qa_questions.items():
    # Aplicamos QA y limpieza solo si el ticket NO es un elogio
    df_tickets[col] = df_tickets.apply(
        lambda row: limpiar_respuesta_qa( # Limpiamos la respuesta...
            safe_answer(row['ticket_text'], pregunta), col  # ...después de ejecutarla con QA
        ) if row['category'] != "Elogio" else "",  # Si es elogio, no analizamos
        axis=1  # Aplicación por fila
    )


In [13]:
# ------------------------------
#  Limpieza Adicional
# ------------------------------

# Si el campo "problema_principal" está vacío o en blanco, lo pasamos a None
df_tickets['problema_principal'] = df_tickets['problema_principal'].apply(
    lambda x: None if not x.strip() else x
)

# Si el campo "nombre_cliente" es igual al texto completo del ticket, probablemente es un error
df_tickets['nombre_cliente'] = df_tickets.apply(
    lambda row: "" if row['nombre_cliente'] == row['ticket_text'] else row['nombre_cliente'],
    axis=1
)


In [14]:
# ------------------------------
#  Visualización de Resultados
# ------------------------------

def mostrar_oraciones(df):
    """
    Muestra una vista humanamente legible del contenido de cada ticket y sus entidades detectadas.
    """
    for i, row in df.iterrows():  # Iteramos sobre cada fila
        print(f"\n *Ticket {i+1}:* ({row['category']})")  # Imprimimos número y categoría
        print(f" {row['ticket_text']}")  # Mostramos el texto completo

        if row['category'] == "Elogio":
            print(" Clasificado como elogio. No se detecta problema.")  # Elogios se ignoran
            print("-" * 80)  # Separador
            continue

        # Mostramos solo si existe contenido útil
        if row['producto_mencionado']:
            print(f" Producto mencionado: {row['producto_mencionado']}")
        if row['ubicacion']:
            print(f" Ubicación: {row['ubicacion']}")
        if row['nombre_cliente']:
            print(f" Cliente: {row['nombre_cliente']}")
        if row['problema_principal']:
            print(f"Problema principal: {row['problema_principal']}")

        print("-" * 80)  # Separador

# Ejecutamos la función de visualización para revisar los resultados
mostrar_oraciones(df_tickets)



 *Ticket 1:* (Problema Técnico)
 Hola, soy Martín Rodríguez de Córdoba. Mi router TP-Link no conecta desde ayer.
 Ubicación: Mi router TP-Link no conecta desde ayer
 Cliente: Martín Rodríguez de Córdoba
Problema principal: Mi router TP-Link no conecta desde ayer
--------------------------------------------------------------------------------

 *Ticket 2:* (Elogio)
 ¡Excelente atención! La asesora Paula me ayudó a configurar mi televisor LG en minutos.
 Clasificado como elogio. No se detecta problema.
--------------------------------------------------------------------------------

 *Ticket 3:* (Problema de Producto)
 Compré una notebook Lenovo en la sucursal Palermo y vino con la pantalla rota.
 Producto mencionado: una notebook
 Ubicación: sucursal Palermo
Problema principal: la pantalla rota
--------------------------------------------------------------------------------

 *Ticket 4:* (Error de Envío)
 ¿Pueden verificar el pedido #89342? Me llegó una cafetera en vez del microondas q

Problemas:

+ El modelo QA interpreta mal la pregunta y devuelve frases completas o irrelevantes.

+ Falta un post-procesamiento más estricto para limpiar respuestas incorrectas.

+ NER no siempre detecta entidades clave (ej: lugares o productos), especialmente si no están bien escritas.

+
No hay control semántico sobre lo que se espera por campo (ej: un producto no puede ser “Tuve un problema”).

Algunas soluciones sugeridas:

+ Mejorar los filtros de limpieza (limpiar_respuesta_qa)
  +  Aplicar validaciones por campo:

          + Productos → deben ser objetos concretos, no frases.
          Ubicación → debe tener forma de lugar, sin verbos.
          Problemas → limitar longitud, cortar frases redundantes.

+ Combinar NER + QA estratégicamente

     + Usar entidades detectadas para validar o completar lo que devuelve QA. Si QA falla, confiar en NER.

*El modelo responde bien, pero necesita supervisión humana codificada: reglas, limpieza y lógica por campo.*



---


**--------------------------------Análisis con Google Gemini--------------------------------**


---



In [17]:
pregunta = f"""Hay más positivos o negativos?

Texto: {df_tickets}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)

Basándonos en la columna "sentiment", hay más comentarios negativos que positivos:

*   **Positivos:** 4
*   **Negativos:** 10

Por lo tanto, hay más comentarios **negativos**.


In [18]:
pregunta = f"""Mencionar el producto de cada ticket, si es que existen, el nombre del cliente, el problema principal

Texto: {df_tickets}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)

Aquí está el desglose de cada ticket, extrayendo el producto (si lo hay), el nombre del cliente y el problema principal:

**Ticket 0:**

*   **Producto:** Router TP-Link
*   **Cliente:** Martín Rodríguez de Córdoba
*   **Problema:** No conecta desde ayer.

**Ticket 1:**

*   **Producto:** No mencionado
*   **Cliente:** No mencionado
*   **Problema:** Elogio

**Ticket 2:**

*   **Producto:** Notebook Lenovo
*   **Cliente:** No mencionado
*   **Problema:** Pantalla rota

**Ticket 3:**

*   **Producto:** Cafetera (recibida incorrectamente en lugar de un microondas)
*   **Cliente:** No mencionado
*   **Problema:** Recibió el producto incorrecto.

**Ticket 4:**

*   **Producto:** Aire acondicionado Samsung
*   **Cliente:** No mencionado
*   **Problema:** Consulta técnica

**Ticket 5:**

*   **Producto:** No mencionado
*   **Cliente:** No mencionado
*   **Problema:** Problema con la entrega en Rosario.

**Ticket 6:**

*   **Producto:** No mencionado
*   **Cliente:** Federico Gómez
*   **Prob

# 4. **Generación de Respuesta Personalizada**



---


**--------------------------------Sugerencias de Respuesta ante Problemas Reportados--------------------------------**


---



In [ ]:
# -----------------------------# Función de respuesta empática -----------------------------
def generar_respuesta_ticket(texto):
    # Frase base de cortesía
    respuesta_inicial = "Estimado cliente, lamentamos mucho lo ocurrido con su pedido."

    # Prompt para Gemini, siguiendo los requisitos del enunciado
    prompt = f"""{texto}

Redactá una respuesta del servicio de atención al cliente que comience así:

"{respuesta_inicial}"

Debe tener un tono amable, local y proactivo. La respuesta no debe superar las 4 líneas.
Debe reconocer el problema y sugerir un siguiente paso (como contactar soporte, derivar el caso, etc.).
"""

    try:
        respuesta = cliente.models.generate_content(
            model=MODEL_ID,
            contents=[prompt]
        )
        # Limpiamos el texto (máx. 4 líneas)
        respuesta_texto = respuesta.text.strip().split("\n")
        return "\n".join(respuesta_texto[:4])

    except Exception as e:
        print(f" Error al generar respuesta: {e}")
        return ""


In [ ]:
# -----------------------------# Aplicar solo a los tickets negativos o con categoría "Problema" -----------------------------

df_tickets['respuesta_cliente'] = df_tickets.apply(
    lambda row: generar_respuesta_ticket(row['ticket_text'])
    if row['sentiment'] == 'Negativo' or 'Problema' in row['category']
    else "",
    axis=1
)


In [ ]:
# -----------------------------# Mostrar respuestas generadas -----------------------------

for _, row in df_tickets.iterrows():
    if row['respuesta_cliente']:
        print("---Ticket---:")
        print(row['ticket_text'])
        print("\n ---Respuesta sugerida---:")
        print(row['respuesta_cliente'])
        print("-" * 80)


---Ticket---:
Hola, soy Martín Rodríguez de Córdoba. Mi router TP-Link no conecta desde ayer.

 ---Respuesta sugerida---:
Estimado cliente, lamentamos mucho lo ocurrido con su pedido. Entendemos la frustración de estar sin internet. Para ayudarte a solucionar el problema con tu TP-Link, te sugerimos que te pongas en contacto con nuestro soporte técnico al 0800-TU-ROUTER (llamada sin cargo desde Córdoba) para una asistencia más personalizada.
--------------------------------------------------------------------------------
---Ticket---:
Compré una notebook Lenovo en la sucursal Palermo y vino con la pantalla rota.

 ---Respuesta sugerida---:
Estimado cliente, lamentamos mucho lo ocurrido con su pedido. ¡Qué garrón lo de la pantalla! Para solucionarlo lo antes posible, por favor, comuníquese con nuestro soporte técnico al 0800-LENVIVO y lo derivaremos con un especialista para el cambio o reparación.
--------------------------------------------------------------------------------
---Ticket

# 5. **Interfaz Interactiva**

In [ ]:
# --------------------- Carga de Modelos ---------------------

# Carga el modelo BETO para análisis de sentimientos en español
beto_sentiment = pipeline("sentiment-analysis", model="finiteautomata/beto-sentiment-analysis")

# Carga el modelo BART para clasificación zero-shot en múltiples categorías
zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Lista de categorías posibles para clasificación. Podés personalizarla según tu dominio
candidate_labels = ['Problema Técnico', 'Elogio', 'Consulta', 'Sugerencia']

# Carga un modelo de NER (Reconocimiento de Entidades) entrenado en español
ner_pipeline = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    aggregation_strategy="simple"  # Agrupa tokens en entidades completas
)

# --------------------- Función para respuesta LLM ---------------------

def generar_respuesta_ticket(texto, sentimiento, categoria):
    """
    Genera una respuesta empática o de agradecimiento, según el tipo de ticket.
    Usa Gemini o un LLM conectado mediante la API.
    """
    # Si el sentimiento es positivo o la categoría es un elogio
    if sentimiento == "Positivo" or categoria == "Elogio":
        respuesta_inicial = "Estimado cliente, muchas gracias por sus amables palabras."

        # Prompt para que el LLM genere una respuesta cálida y positiva
        prompt = f"""{texto}

Redactá una respuesta del servicio de atención al cliente que comience así:

"{respuesta_inicial}"

Debe tener un tono cálido, cercano y positivo. Agradecé la valoración y ofrecé seguir acompañando si es necesario. No más de 4 líneas.
"""
    else:
        # Caso negativo o problema: generar una disculpa y acción
        respuesta_inicial = "Estimado cliente, lamentamos mucho lo ocurrido con su pedido."

        # Prompt para que el LLM genere una respuesta empática y resolutiva
        prompt = f"""{texto}

Redactá una respuesta del servicio de atención al cliente que comience así:

"{respuesta_inicial}"

Debe tener un tono amable, local y proactivo. La respuesta no debe superar las 4 líneas.
Debe reconocer el problema y sugerir un siguiente paso (como contactar soporte, derivar el caso, etc.).
"""
    try:
        # Llamada al modelo LLM (Gemini o similar) para generar la respuesta
        respuesta = cliente.models.generate_content(
            model=MODEL_ID,
            contents=[prompt]
        )

        # Se limpia la respuesta para devolver solo las primeras 4 líneas
        respuesta_texto = respuesta.text.strip().split("\n")
        return "\n".join(respuesta_texto[:4])

    except Exception as e:
        # En caso de error en la API, se devuelve un mensaje de fallback
        print(f"Error al generar respuesta: {e}")
        return "Lo sentimos, hubo un error al generar la respuesta."


# --------------------- Función Principal de Inferencia ---------------------

def inferir_ticket(ticket_text):
    """
    Ejecuta todo el pipeline de inferencia sobre un ticket nuevo:
    sentimiento, categoría, entidades y respuesta generada.
    """
    # Predice el sentimiento del ticket (Positivo o Negativo)
    sentimiento = beto_sentiment(ticket_text)[0]['label']

    # Clasifica el ticket en una categoría usando zero-shot
    categoria = zero_shot(ticket_text, candidate_labels=candidate_labels)['labels'][0]

    # Extrae entidades nombradas del texto (ubicaciones, nombres, productos, etc.)
    entidades = ner_pipeline(ticket_text)
    entidades_resultado = ', '.join([ent['word'] for ent in entidades])  # Se formatea como string plano

    # Genera una respuesta del LLM según el análisis anterior
    respuesta = generar_respuesta_ticket(ticket_text, sentimiento, categoria)

    # Devuelve todos los resultados a la interfaz
    return sentimiento, categoria, entidades_resultado, respuesta


# --------------------- Interfaz Gradio ---------------------

# Se crea la interfaz visual con Gradio
iface = gr.Interface(
    fn=inferir_ticket,  # Función principal que corre todo el pipeline
    inputs=gr.Textbox(  # Campo de entrada donde se escribe el ticket
        label="Ingresa tu ticket de soporte",
        placeholder="Escribe el ticket aquí..."
    ),
    outputs=[  # Cuatro salidas: sentimiento, categoría, entidades, respuesta
        gr.Textbox(label="Sentimiento Predicho"),
        gr.Textbox(label="Categoría Predicha"),
        gr.Textbox(label="Entidades/Respuesta Clave"),
        gr.Textbox(label="Respuesta Generada por LLM")
    ],
    live=True  # Actualiza dinámicamente mientras se escribe
)

# Lanzamiento de la aplicación web local
iface.launch()


Device set to use cpu


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://681c45dff17689a851.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
